In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import time
import re
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen
import pprint
import openpyxl

In [2]:
#selenium前準備
path="見せられないヨ" #chromedriverのpath

In [3]:
#会社を舶用工業会でスクレイピング
chrome = webdriver.Chrome(executable_path=path)
chrome.get("https://www.jsmea.or.jp/directory/member-j.php")

html = chrome.page_source.encode('utf-8')
soup= BeautifulSoup(html, 'html.parser')

org_list=[] #企業名を格納
boolean=True
counter=0

#企業名を全て収集したら終わり
while boolean:
    try:
        organization=soup.find_all('td', {'width':'36%'})[counter].get_text().replace('（株）', '').replace('(株)', '')
        org=re.sub('[ぁ-ん]*行',"", organization)
        org_list.append(org)
        
    except IndexError:
        boolean=False
    
    counter=counter+1

chrome.quit()
org_list = list(filter(len, org_list))
print(org_list)
print()
print("見つかった企業数は{}".format(len(org_list)))
#org_list.append(org)

['ＩＨＩ', 'ＩＨＩ回転機械エンジニアリング', 'ＩＨＩ原動機', 'ＡＮＥＯＳ', 'ＡＭＣＯエンジニアリング', 'アイエスエスマシナリーサービスリミテッド', 'アイメックス', 'アズビル', 'アルファ・ラバル', '相浦機械', '赤阪鐵工所', '旭化成エンジニアリング', '東化工', 'イーグル工業', 'イソダメタル', 'イワキテック', '池貝ディーゼル', '石井工作所', 'いすゞ自動車', '伊吹工業', 'ウインターツールガスアンドディーゼルジャパン', 'ウエイクフィールド', 'ウッズ', '潮冷熱', '宇津木計器', '宇都宮電機製作所', 'ＳＫＫ', 'ＳＫウインチ\u3000', 'エクセノヤマミズ', 'エヌゼットケイ', 'エヌワイ', 'エムエス工業', '江真トレーディング', 'オーケーエム', 'オリジン', 'オンセック', '大石電機工業', '大阪アサヒメタル工場', '大阪送風機製作所', '大阪ボイラー製作所', '岡本鉄工', 'ガイスリンガー', 'カシワテック', '神奈川機器工業', '金澤鐵工', '鐵萬商事', '神威産業', 'かもめプロペラ', '川崎重工業', '神崎高級工機製作所', '岸上バルブ', '北澤電機製作所', '京泉工業', '共立機械製作所', '共和工業', '極洋電機', 'クラレ', '國森', '久保田工業', '倉本計器精工所', '倉本産業', '桑畑電機', 'ケーイーアイシステム', 'ケイ アンド ケイ', 'ＣＯＡＳＴ', 'コーアディーゼル', 'コスモ', 'コンシリアム・ニッタンマリーン', 'コンヒラ', '高工社', '神戸機材', '神戸製鋼所', '国際化工', '五光製作所', '小坂研究所', '金剛コルメット製作所', 'サクション瓦斯機関製作所', 'ササクラ', 'サタケ', 'サンフレム', '佐世保重工業', '三協製作所', '三工電機', '三信船舶電具', '三和商事', '三和鉄工', 'ＪＲＣＳ', 'ＪＦＥエンジニアリング', 'シンコー', 'ジャパンエンジンコーポレーション', 'ジャパン・ハムワージ', '品川重工', '島田燈器工業', '商船三井テクノトレード', '湘南工作所', '湘洋

# 以下はyahooファイナンス内をクローリング

In [4]:
#yahooファイナンスで造船会社を検索してurlを取得
chrome = webdriver.Chrome(executable_path=path)
chrome.get('https://finance.yahoo.co.jp/')

linklist=[]
for keyword in org_list:
    element = chrome.find_element_by_name("query")
    element.send_keys(keyword)
    element.send_keys(Keys.RETURN)
    chrome.find_element_by_name("query").clear()
    time.sleep(1)
    
    #urlを取得
    htmltext1 = chrome.page_source.encode('utf-8')
    try: #企業情報が一つもなかった場合例外処理
        soup1= BeautifulSoup(htmltext1, 'html.parser')
        get_url=soup1.find("li",{"class":"_239Zl3PI"}).find('a').attrs['href']
        
        #正規表現で外国株を排除
        matchurl=re.match("https://finance.yahoo.co.jp/quote/*[0-9]*.T",get_url)
        if matchurl:
            print("{}は成功".format(keyword))
            print(get_url)
            linklist.append(get_url)
        else:
            print("{}は日本株ではありません".format(keyword))
            print(get_url)
            org_list.remove(keyword)

    except AttributeError:
        print("{}は失敗".format(keyword))
        org_list.remove(keyword)
        
print("リンクが取得出来た企業数は{}".format(len(get_url)))

ＩＨＩは成功
https://finance.yahoo.co.jp/quote/7013.T
ＩＨＩ回転機械エンジニアリングは失敗
ＡＮＥＯＳは失敗
アイエスエスマシナリーサービスリミテッドは失敗
アズビルは成功
https://finance.yahoo.co.jp/quote/6845.T
アルファ・ラバルは失敗
赤阪鐵工所は成功
https://finance.yahoo.co.jp/quote/6022.T
旭化成エンジニアリングは失敗
イーグル工業は成功
https://finance.yahoo.co.jp/quote/6486.T
イソダメタルは失敗
池貝ディーゼルは失敗
いすゞ自動車は成功
https://finance.yahoo.co.jp/quote/7202.T
伊吹工業は失敗
ウエイクフィールドは失敗
潮冷熱は失敗
宇都宮電機製作所は失敗
ＳＫウインチ　は失敗
エヌゼットケイは失敗
エムエス工業は失敗
オーケーエムは成功
https://finance.yahoo.co.jp/quote/6229.T
オリジンは成功
https://finance.yahoo.co.jp/quote/6513.T
オンセックは失敗
大阪アサヒメタル工場は失敗
大阪ボイラー製作所は失敗
ガイスリンガーは失敗
神奈川機器工業は失敗
鐵萬商事は失敗
かもめプロペラは失敗
神崎高級工機製作所は失敗
北澤電機製作所は失敗
共立機械製作所は失敗
極洋電機は失敗
國森は失敗
倉本計器精工所は失敗
桑畑電機は失敗
ケイ アンド ケイは成功
https://finance.yahoo.co.jp/quote/3760.T
ＣＯＡＳＴは日本株ではありません
https://stocks.finance.yahoo.co.jp/us/detail/ACAB
コスモは成功
https://finance.yahoo.co.jp/quote/3349.T
コンシリアム・ニッタンマリーンは失敗
高工社は失敗
神戸製鋼所は成功
https://finance.yahoo.co.jp/quote/5406.T
国際化工は失敗
小坂研究所は失敗
サクション瓦斯機関製作所は失敗
サタケは成功
https://finance.yahoo.co.jp/quote/1826.T
サンフレムは失敗

In [5]:
#urlで検索するメソッド
def get_info(url):
    html = urlopen(url) #BeautifulSoupで抽出
    bs = BeautifulSoup(html, 'html.parser')
    
    #会社名
    finder=bs.find("section",{"class":"_1zZriTjI _2l2sDX5w"}).find("h1",{"class":"_6uDhA-ZV"}).get_text()
    #会社名
    print(finder)

    #平均年収
    finder1=bs.find("tbody").find_all("tr")[15].find("td",{"class":"_1Y-2cHme"})
    get_income=finder1.get_text()
    income=int(re.findall('[0-9]+',get_income.replace(",",""))[0])*1000/10000
    print("{}万円".format(income))

    #平均年齢
    finder2=bs.find("tbody").find_all("tr")[14].find("td",{"class":"_1Y-2cHme"})
    get_age=finder2.get_text()
    age=float(re.findall("\d+\.\d+",get_age)[0])
    print("{}才".format(age))
    print(url)

In [6]:
#yahooで企業情報を検索するurl
for linkadd in linklist:
    try:
        url=linkadd+"/profile"
        get_info(url)
        time.sleep(2) #スクレイピング間隔
        print()
    except HTTPError:
        print("企業情報もらえなかったアヘアへv(ﾟ∀｡)v")
chrome.quit()

(株)ＩＨＩ
736.0万円
40.8才
https://finance.yahoo.co.jp/quote/7013.T/profile

アズビル(株)
773.0万円
45.9才
https://finance.yahoo.co.jp/quote/6845.T/profile

(株)赤阪鐵工所
455.0万円
41.4才
https://finance.yahoo.co.jp/quote/6022.T/profile

イーグル工業(株)
763.0万円
41.1才
https://finance.yahoo.co.jp/quote/6486.T/profile

いすゞ自動車(株)
753.0万円
41.2才
https://finance.yahoo.co.jp/quote/7202.T/profile

(株)オーケーエム
526.0万円
40.7才
https://finance.yahoo.co.jp/quote/6229.T/profile

(株)オリジン
633.0万円
44.8才
https://finance.yahoo.co.jp/quote/6513.T/profile

(株)ケイブ
480.0万円
35.9才
https://finance.yahoo.co.jp/quote/3760.T/profile

(株)コスモス薬品
420.0万円
30.4才
https://finance.yahoo.co.jp/quote/3349.T/profile

(株)神戸製鋼所
546.0万円
38.9才
https://finance.yahoo.co.jp/quote/5406.T/profile

佐田建設(株)
544.0万円
45.7才
https://finance.yahoo.co.jp/quote/1826.T/profile

(株)ジャパンエンジンコーポレーション
578.0万円
41.2才
https://finance.yahoo.co.jp/quote/6016.T/profile

昭和産業(株)
691.0万円
39.7才
https://finance.yahoo.co.jp/quote/2004.T/profile

ダイハツディーゼル(株)
590.0万円
41.3才
https://finance.y

In [7]:
print(len(linklist))
print(len(org_list))
for i in range(len(org_list)):
    try:
        print(linklist[i])
    except IndexError:
        print()
    
    try:
        print(org_list[i])
    except IndexError:
        print()
#chrome.quit()

28
135
https://finance.yahoo.co.jp/quote/7013.T
ＩＨＩ
https://finance.yahoo.co.jp/quote/6845.T
ＩＨＩ原動機
https://finance.yahoo.co.jp/quote/6022.T
ＡＭＣＯエンジニアリング
https://finance.yahoo.co.jp/quote/6486.T
アイメックス
https://finance.yahoo.co.jp/quote/7202.T
アズビル
https://finance.yahoo.co.jp/quote/6229.T
相浦機械
https://finance.yahoo.co.jp/quote/6513.T
赤阪鐵工所
https://finance.yahoo.co.jp/quote/3760.T
東化工
https://finance.yahoo.co.jp/quote/3349.T
イーグル工業
https://finance.yahoo.co.jp/quote/5406.T
イワキテック
https://finance.yahoo.co.jp/quote/1826.T
石井工作所
https://finance.yahoo.co.jp/quote/6016.T
いすゞ自動車
https://finance.yahoo.co.jp/quote/2004.T
ウインターツールガスアンドディーゼルジャパン
https://finance.yahoo.co.jp/quote/6023.T
ウッズ
https://finance.yahoo.co.jp/quote/5471.T
宇津木計器
https://finance.yahoo.co.jp/quote/7245.T
ＳＫＫ
https://finance.yahoo.co.jp/quote/4617.T
エクセノヤマミズ
https://finance.yahoo.co.jp/quote/7721.T
エヌワイ
https://finance.yahoo.co.jp/quote/6461.T
江真トレーディング
https://finance.yahoo.co.jp/quote/6490.T
オーケーエム
https://finance.yahoo.co.jp